In [1]:
# Imports

# Internal
import db_helpers as dbh
import db_parse as dbp

# External
import ipywidgets as widgets
import json
import pandas as pd
from pprint import pprint
import os

In [2]:
# Settings

db_name = "dos"
schema = "schema.sql"
data_dir = "/usr/local/dos-mitigation/data"

In [ ]:
# Select Materialization

subfolders = [ f.path for f in os.scandir(data_dir) if f.is_dir() ]
materializations = [i.split('/')[-1] for i in subfolders]
widget_materialization = widgets.Dropdown(
    options=materializations,
    value=materializations[0],
    description='Materialization',
    disabled=False
)
widget_materialization

In [ ]:
# Select Session

materialization = widget_materialization.value
revision = materialization
materialization_dir = data_dir + '/' + materialization
subfolders = [ f.path for f in os.scandir(materialization_dir) if f.is_dir() ]
sessions = [i.split('/')[-1] for i in subfolders]
widget_session = widgets.Dropdown(
    options=sessions,
    value=sessions[0],
    description='Session',
    disabled=False
)
widget_session

In [7]:
# Check for Existing Data

session = widget_session.value
session_dir = materialization_dir + '/' + session
conn = dbh.Connection("dos")

try:
    existing_revisions = [i[0] for i in conn.db_query("select distinct nickname from revisions")]
except IndexError:
    existing_revisions = []

if revision not in existing_revisions:
    conn.insert_dict_as_row('revisions', {
        "hash": "5bcc425177d073587dc0f78bf9acf084676d2061",
        "metal": True,
        "model_file": None,
        "nickname": revision
    })
    print(f"Added revision with nickname {revision}")
revision_id = conn.nickname_id("revisions", revision)

try:
    existing_materializations = [i[0] for i in conn.db_query(f"select distinct nickname from materializations where revision = {revision_id}")]
except IndexError:
    existing_materializations = []

if materialization not in existing_materializations:
    conn.insert_dict_as_row('materializations', {
        "nickname": materialization,
        "revision": revision_id,
        "inventory_file": None
    })
    print(f"Added materialization with nickname {materialization}")
materialization_id = conn.nickname_id("materializations", materialization)

try:
    existing_sessions = conn.db_query(f"select distinct nickname from sessions where materialization = {materialization_id}")[0]
except IndexError:
    existing_sessions = []

widget_existing_session = None
if session in existing_sessions:
    print(f'WARNING: Session {session} already exists')
    widget_existing_session = widgets.Dropdown(
        options=['Append Nickname', 'Merge and Keep New Parameters', 'Merge and Keep Old Parameters'],
        value = 'Append Nickname',
        description = 'Action:',
        disabled=False
    )
widget_existing_session


In [ ]:
# Populate Session Metadata

session_nickname = session
if widget_existing_session != None:
    existing_session_action = widget_existing_session.value
    if existing_session_action == 'Append Nickname':
        i = 0
        while session + str(i) in existing_sessions:
            i += 1
        session_nickname = session + str(i)
        print(f'Appending session nickname from {session} to {session_nickname}')
        with open("{}/.parameters.json".format(session_dir)) as f:
            parameters = json.load(f)
        conn.insert_dict_as_row("sessions", {
            "materialization": materialization_id,
            "parameters": parameters,
            "nickname": session_nickname
        })
        print(f"Added session with nickname {session + str(i)}")

    elif existing_session_action == 'Merge and Keep New Parameters':
        with open("{}/.parameters.json".format(session_dir)) as f:
            parameters = json.load(f)
        conn.insert_dict_as_row("sessions", {
            "materialization": materialization_id,
            "parameters": parameters,
            "nickname": session_nickname
        })
        print(f"Added session with nickname {session_nickname}")
    else:
        pass
else:
    with open("{}/.parameters.json".format(session_dir)) as f:
        parameters = json.load(f)
    conn.insert_dict_as_row("sessions", {
        "materialization": materialization_id,
        "parameters": parameters,
        "nickname": session
    })
    print(f"Added session with nickname {session_nickname}")


session_id = conn.nickname_id("sessions", session_nickname)

In [ ]:
# Parse Session Experiments

experiment_list = os.listdir("{}".format(session_dir))
experiment_list = [i for i in experiment_list if not i.startswith('.')]
print(f"Found {len(experiment_list)} experiments")
i=1
for experiment in experiment_list:
    if experiment.startswith('.'):
        continue
    print(f"{i}/{len(experiment_list)}: ", end='')
    dbp.parse_experiment(conn, materialization, session, experiment)
    dbp.analyze_experiment(conn, experiment)
    i += 1